## Deliverable 2. Create a Customer Travel Destinations Map.

In [16]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [19]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Constitucion,CL,-35.3333,-72.4167,59.52,57,37,5.46,scattered clouds
1,1,Hasaki,JP,35.7333,140.8333,70.29,89,56,9.19,broken clouds
2,2,Saskylakh,RU,71.9167,114.0833,18.48,96,100,5.53,overcast clouds
3,3,Kodiak,US,57.7900,-152.4072,45.84,76,18,11.79,few clouds
4,4,Hermanus,ZA,-34.4187,19.2345,62.80,83,82,11.18,light rain


In [20]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 100


In [21]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
17,17,Lynn Haven,US,30.2455,-85.6483,87.91,64,75,3.00,broken clouds
19,19,Vaini,TO,-21.2000,-175.2000,80.76,83,75,4.61,broken clouds
21,21,Kapaa,US,22.0752,-159.3190,87.78,68,75,5.01,broken clouds
22,22,San Patricio,US,28.0170,-97.5169,90.01,39,0,9.82,clear sky
38,38,Natal,BR,-5.7950,-35.2094,83.05,74,40,16.11,scattered clouds
43,43,Butaritari,KI,3.0707,172.7902,82.54,70,37,12.68,scattered clouds
45,45,Touros,BR,-5.1989,-35.4608,83.30,76,99,14.99,overcast clouds
50,50,North Platte,US,41.1239,-100.7654,84.09,31,1,1.99,clear sky
56,56,Georgetown,MY,5.4112,100.3354,80.89,91,20,0.00,few clouds
58,58,Hirara,JP,24.8000,125.2833,82.35,71,100,18.92,overcast clouds


In [22]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [24]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# As evidenced above, this step is N/A.

In [26]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
17,Lynn Haven,US,87.91,broken clouds,30.2455,-85.6483,
19,Vaini,TO,80.76,broken clouds,-21.2000,-175.2000,
21,Kapaa,US,87.78,broken clouds,22.0752,-159.3190,
22,San Patricio,US,90.01,clear sky,28.0170,-97.5169,
38,Natal,BR,83.05,scattered clouds,-5.7950,-35.2094,
43,Butaritari,KI,82.54,scattered clouds,3.0707,172.7902,
45,Touros,BR,83.30,overcast clouds,-5.1989,-35.4608,
50,North Platte,US,84.09,clear sky,41.1239,-100.7654,
56,Georgetown,MY,80.89,few clouds,5.4112,100.3354,
58,Hirara,JP,82.35,overcast clouds,24.8000,125.2833,


In [28]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")     

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [45]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   150
Country                150
Max Temp               150
Current Description    150
Lat                    150
Lng                    150
Hotel Name             150
dtype: int64

In [50]:
# 8a. Create the output File (CSV)
# Create the output file (CSV).
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [56]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [52]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))